# Credit Portfolio Analysis

## Introduction
This Jupyter notebook is designed to perform a **comprehensive credit portfolio analysis** on the provided loan and payment datasets. The goal is to compute key **portfolio quality and performance metrics** that are critical for risk management and financial reporting. Specifically, the notebook calculates:  

- **Default Rate** – proportion of loans that have become non-performing (past due > 120 days and not yet settled)  
- **Write-off Rate** – share of total principal written off due to default  
- **Portfolio at Risk (PAR)** – outstanding balance of loans overdue beyond a specified threshold relative to the total active portfolio  
- **Recovery Rate** – effectiveness of recovering amounts from defaulted loans  
- **Average Days Past Due (DPD)** – typical delay in repayment among overdue loans  
- **Net Interest Margin (NIM) per Loan** – simplified gross yield including interest and net fees  
- **Loan Loss Provisioning** – estimated provisioning expense based on overdue loans and company policy  

---

## Notebook Structure

### 1. Import Necessary Libraries
- import all libraries and module need for the task
### 2. Data Loading
- Load the **loan dataset** containing loan details such as principal, interest, fees, due dates, and settlement dates.
- Load the **payment dataset** containing all repayments made by customers with multiple payment records per loan.

### 3. Data Cleaning & Preparation
- Convert date columns to **datetime** format.
- Aggregate payments per loan to compute **total amount paid**.

### 4. KPI Calculations
- **Default Rate** – count loans overdue >120 days and not settled.
- **Write-off Rate** – proportion of principal for written-off loans.
- **Portfolio at Risk (PAR)** – outstanding balance of overdue loans relative to total active portfolio.
- **Recovery Rate** – recovered amounts from defaulted loans as a percentage of total defaulted amount.
- **Average DPD** – mean days past due for active past-due loans.
- **Net Interest Margin (NIM)** – calculated per loan based on earned interest, net fees, and loan tenure.
- **Loan Loss Provisioning** – compute provision per loan based on DPD and total portfolio provisioning cost.


### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date

### Load the Dataset

In [2]:
# read loan data
loan_data = pd.read_excel("loans_data.xlsx")

In [3]:
# Display sample load data
loan_data.head(3)

,loan_code,customer_id,product_type,disbursed_date,disbursed_amount,loan_due_date,settlement_date,gross_loan_interest,surplus_payment,unerned_interest,penalties,waiver,initiation_fee,admin_fee,appointment_fee,joining_fee,fix,total_due
0,-8063280106777311930,8.138699e+18,CASH_SALES,2018-04-03,1250,2018-05-02,2018-04-03 00:00:00 UTC,0.0,0.0,0.0,0,-1250.0,0,0.0,0.0,0.0,0.0,0.0
1,-3785456775112180505,8.138699e+18,CASH_SALES,2018-01-23,1250,2018-02-22,2018-01-23 00:00:00 UTC,0.0,0.0,0.0,0,-1250.0,0,0.0,0.0,0.0,0.0,0.0
2,4192675035644252092,8.138699e+18,CASH_SALES,2018-08-23,90000,2018-09-24,2018-08-27 00:00:00 UTC,0.3,0.0,0.0,0,-30000.0,0,0.0,0.0,0.0,0.0,87000.0


In [4]:
# read payment data
payment_data = pd.read_excel("payments_data.xlsx")

In [5]:
# Display sample payment data
payment_data.head()

,date,loan_rep_code,amount,id
0,2019-12-11,6551093366263666623,97,-3925191339544160207
1,2019-08-18,2538002466878584691,6000,7545899052503010380
2,2019-08-19,2538002466878584691,2560,2168229508398644566
3,2019-05-28,-7091408425690813765,24242,8115022827661941455
4,2020-01-02,5220885072334740083,9900,-3667170369671842832


In [6]:
# Check features available in loan data
print(f"=== Here are variables present in Loan data ===")
print(loan_data.columns)
print(f"\n")

# Display features available in pyament data
print(f"=== Here are variables present in Payment data === ")
print(payment_data.columns)

=== Here are variables present in Loan data ===
Index(['loan_code', 'customer_id', 'product_type', 'disbursed_date',
       'disbursed_amount', 'loan_due_date', 'settlement_date',
       'gross_loan_interest', 'surplus_payment', 'unerned_interest',
       'penalties', 'waiver', 'initiation_fee', 'admin_fee', 'appointment_fee',
       'joining_fee', 'fix', 'total_due'],
      dtype='object')


=== Here are variables present in Payment data === 
Index(['date', 'loan_rep_code', 'amount', 'id'], dtype='object')


### Data Preparation

In [7]:
## confirm of variables in loan data has appriate data type to allow operation to be performed on them
loan_data.dtypes

loan_code                       int64
customer_id                   float64
product_type                   object
disbursed_date         datetime64[ns]
disbursed_amount                int64
loan_due_date          datetime64[ns]
settlement_date                object
gross_loan_interest           float64
surplus_payment               float64
unerned_interest              float64
penalties                       int64
waiver                        float64
initiation_fee                  int64
admin_fee                     float64
appointment_fee               float64
joining_fee                   float64
fix                           float64
total_due                     float64
dtype: object

In [8]:
## convert settlement_date to python date object
loan_data['settlement_date'] = pd.to_datetime(loan_data['settlement_date']).dt.normalize()

In [9]:
## check and handle data types issue on payment data
payment_data.dtypes

date             datetime64[ns]
loan_rep_code             int64
amount                    int64
id                        int64
dtype: object

### Aggregate payments per loan

In [10]:
# peform aggregation to get total repayment amount of each loan and the last paynment data
payments_agg = payment_data.groupby('loan_rep_code', as_index=False).agg(
    total_amount_paid=('amount', 'sum'),
    last_payment_date=('date', 'max')
).fillna(0)

## handle null values in  total_amount_paid column
payments_agg["total_amount_paid"] = payments_agg["total_amount_paid"].fillna(0)

# Display sample records
payments_agg.head(3)

,loan_rep_code,total_amount_paid,last_payment_date
0,-9223351916581235936,435600,2022-11-07
1,-9223347553164632852,358500,2024-07-06
2,-9223306595261143748,50000,2022-09-21


### Merge loan and payment data
To make the computation requested we need to merge both two dataset

In [11]:
# merge with loan data
loan_payment_df = loan_data.merge(
    payments_agg,
    left_on='loan_code',     # column name in loan_data
    right_on='loan_rep_code', # column name in payments_agg
    how='left'                # consider all entries from the left table and only the matching from the right only
).drop(columns=['loan_rep_code']) # drop loan_rep_code to avoid duplication of the column



# Display sample row from the merged dataset
loan_payment_df.head(2)

,loan_code,customer_id,product_type,disbursed_date,disbursed_amount,loan_due_date,settlement_date,gross_loan_interest,surplus_payment,unerned_interest,penalties,waiver,initiation_fee,admin_fee,appointment_fee,joining_fee,fix,total_due,total_amount_paid,last_payment_date
0,-8063280106777311930,8.138699e+18,CASH_SALES,2018-04-03,1250,2018-05-02,2018-04-03 00:00:00+00:00,0.0,0.0,0.0,0,-1250.0,0,0.0,0.0,0.0,0.0,0.0,5509400.0,2023-08-27
1,-3785456775112180505,8.138699e+18,CASH_SALES,2018-01-23,1250,2018-02-22,2018-01-23 00:00:00+00:00,0.0,0.0,0.0,0,-1250.0,0,0.0,0.0,0.0,0.0,0.0,428000.0,2023-08-31


### Compute Requested Metrics

#### 1. Default Rate
proportion of loans that have become non-performing (past due > 120 days and not yet settled)  

In [12]:
# DPD (days past due, only for active loans)
loan_payment_df['day_past_due'] = (loan_payment_df['last_payment_date'] - loan_payment_df['loan_due_date']).dt.days

# Ensure 'day_past_due' column has no negative values and is stored as integer
loan_payment_df['day_past_due'] = (
    loan_payment_df['day_past_due']       
    .clip(lower=0)                        # set all negative values to 0
    .astype("Int64")                      # convert to pandas nullable integer type (preserves NaN)
)


# create a flag for a default loan only for active load
loan_payment_df['is_default'] = (loan_payment_df['day_past_due'] > 120) & loan_payment_df['settlement_date'].isna()

# Compute the default rate
default_rate = (loan_payment_df['is_default'].sum() / len(loan_payment_df)) * 100
print(f"The current default rate for the portfolio is {default_rate:.2f}% of total loans.")


The current default rate for the portfolio is 2.45% of total loans.


#### 2. Write-off Rate (Due Date +120 days)
This represent share of total principal written off due to default  

In [13]:
# Calculate the total principal amount of loans that are written off
# Loans are considered written off if days past due (dpd) > 120
total_principal_written_off = loan_payment_df.loc[loan_payment_df['day_past_due'] > 120, 'disbursed_amount'].sum()

# Calculate the total principal of all loans
total_principal_all = loan_payment_df['disbursed_amount'].sum()

# Compute the write-off rate as a percentage of total principal
write_off_rate = (total_principal_written_off / total_principal_all) * 100

print(f"The write-off rate of the portfolio is {write_off_rate:.2f}% of the total disbursed principal.")

The write-off rate of the portfolio is 8.34% of the total disbursed principal.


#### 3. Portfolio at Risk (PAR &gt; 30,60,90 days)
Outstanding balance of loans overdue beyond a specified threshold relative to the total active portfolio

In [14]:
# Portfolio at Risk (PAR) Calculation

# Step 1: Calculate outstanding balance for each loan
loan_payment_df['outstanding'] = loan_payment_df['total_due'] - loan_payment_df['total_amount_paid']

# Step 2: Filter only active loans (settlement_date is NULL)
active_loans = loan_payment_df[loan_payment_df['settlement_date'].isna()]

# Step 3: Calculate total outstanding balance for all active loans
total_outstanding = active_loans['outstanding'].sum()

# Step 4: Define PAR thresholds in days
par_thresholds = [30, 60, 90]
par_results = {}

# Step 5: Compute PAR for each threshold
for days in par_thresholds:
    # Sum of outstanding balances for loans past due greater than the threshold
    outstanding_overdue = active_loans.loc[active_loans['day_past_due'] > days, 'outstanding'].sum()
    
    # Calculate PAR as a percentage of total outstanding
    par_percentage = (outstanding_overdue / total_outstanding) * 100
    
    # Store result
    par_results[days] = par_percentage

# Step 6: Print PAR results
for days, par_value in par_results.items():
    print(f"Portfolio at Risk (PAR) for loans overdue more than {days} days: {par_value:.2f}%")


Portfolio at Risk (PAR) for loans overdue more than 30 days: 152.88%
Portfolio at Risk (PAR) for loans overdue more than 60 days: 182.91%
Portfolio at Risk (PAR) for loans overdue more than 90 days: 202.10%


#### 4. Recovery Rate
this respresent effectiveness of recovering amounts from defaulted loans

In [15]:
# Calculate the total amount recovered from defaulted loans
# Only consider loans marked as defaulted (is_default == True)
# Sum up all payments made towards these defaulted loans
recovered_from_defaults = (
    loan_payment_df.loc[loan_payment_df['is_default'], 'total_amount_paid'].sum()
)

# Calculate the total amount in default
# Using total_due for each defaulted loan to represent the full amount owed at the time of default
total_default_amount = loan_payment_df.loc[loan_payment_df['is_default'], 'total_due'].sum()

# Calculate Recovery Rate as a percentage
# Recovery Rate = (Amount Recovered) / (Total Amount in Default) * 100
# If no defaults exist  set recovery rate to 0 to avoid division by zero error
recovery_rate = (recovered_from_defaults / total_default_amount) * 100 if total_default_amount > 0 else 0
print(f"The recovery rate from defaulted loans is {recovery_rate:.2f}% of the total amount in default.")

The recovery rate from defaulted loans is 253.08% of the total amount in default.


#### 5. Average Days Past Due (DPD)
typical delay in repayment among overdue loans

In [16]:
# Filter only active loans that are past due (Current Date > loan_due_date)
past_due_loans = loan_payment_df[(loan_payment_df['settlement_date'].isna()) & 
                             (loan_payment_df['last_payment_date'] > loan_payment_df['loan_due_date'])]


# Average DPD = sum of DPD / number of past-due loans
average_dpd = past_due_loans['day_past_due'].sum() / len(past_due_loans) if len(past_due_loans) > 0 else 0

print(f"The average days past due (DPD) for active past-due loans is {average_dpd:.2f} days.")


The average days past due (DPD) for active past-due loans is 355.34 days.


#### 6. Net Interest Margin (NIM) per Loan
simplified gross yield including interest and net fees

In [30]:
# Step 1: Compute effective settlement date (fallback to last payment if settlement is missing)
loan_payment_df['effective_settlement_date'] = loan_payment_df['settlement_date'].fillna(loan_payment_df['last_payment_date'])

loan_payment_df['effective_settlement_date'] = pd.to_datetime(
    loan_payment_df['effective_settlement_date'], errors='coerce'
).dt.tz_localize(None).dt.normalize()

# Step 2: Calculate earned interest
loan_payment_df['earned_interest'] = loan_payment_df['gross_loan_interest'] - loan_payment_df['unerned_interest']

# Step 3: Calculate net fees: sum of all fees minus waivers
loan_payment_df['net_fees'] = (
    loan_payment_df[['initiation_fee','admin_fee','appointment_fee','joining_fee','fix']].sum(axis=1) - loan_payment_df['waiver']
)

# Step 4: Compute loan tenure in years
loan_payment_df['loan_tenure_years'] = (
    (loan_payment_df['effective_settlement_date'] - loan_payment_df['disbursed_date']).dt.days / 365
)

# Step 5: Compute denominator for NIM
denominator = loan_payment_df['disbursed_amount']

# initialize disbursement cost
disbursement_cost = 70

# Step 6: Compute NIM, set to 0 if denominator <= 0 to avoid inf
loan_payment_df['net_interest_margin'] = np.where(
    denominator > 0,
    (loan_payment_df['earned_interest'] + ((loan_payment_df['net_fees']) - disbursement_cost)) / denominator,
    0  # set NIM to 0 for zero tenure or zero disbursed amount
)

# Step 7: Display first 5 loans
print("NIM per loan (first 5 loans):")
loan_payment_df[['loan_code','settlement_date','disbursed_date','net_interest_margin']].head()


NIM per loan (first 5 loans):


,loan_code,settlement_date,disbursed_date,net_interest_margin
0,-8063280106777311930,2018-04-03 00:00:00+00:00,2018-04-03,0.944000
1,-3785456775112180505,2018-01-23 00:00:00+00:00,2018-01-23,0.944000
2,4192675035644252092,2018-08-27 00:00:00+00:00,2018-08-23,0.332559
3,-6556919864984443676,2018-12-10 00:00:00+00:00,2018-11-27,0.159226
4,1796654104776969268,2019-01-05 00:00:00+00:00,2018-12-20,0.139226


#### 7. Loan Loss Provisioning &amp; Cost Impact
Estimated provisioning expense based on overdue loans and company policy 

In [ ]:
# Define provisioning rate based on DPD
def provisioning_rate(dpd):
    if dpd <= 30:
        return 0.05  # 5%
    elif dpd <= 120:
        return 0.25  # 25%
    else:
        return 1.0   # 100% for defaulted loans

loan_payment_df['provision_rate'] = loan_payment_df['day_past_due'].apply(provisioning_rate)

# Provision per loan
loan_payment_df['loan_provision'] = loan_payment_df['outstanding'] * loan_payment_df['provision_rate']

# Total provision expense for the portfolio
total_provision_expense = loan_payment_df['loan_provision'].sum()

print(f"The total loan loss provision expense for the portfolio is {total_provision_expense:.2f}.")


The total loan loss provision expense for the portfolio is -49920069769.45.


## Conclusion

The analysis of the loan portfolio reveals a **relatively low default rate** of 2.45% of total loans, while the **write-off rate** stands at 8.34% of the total disbursed principal. However, the **Portfolio at Risk (PAR > 30,60,and 90 days)** is 152.88%,182.91%, and 202.10% respectively. The **recovery rate** exceeds 100% at 253.08%, suggesting that collections from defaulted loans have more than covered the amounts classified as in default, potentially due to overpayments or data inconsistencies. The **average DPD** for active past-due loans is 355.34 days, highlighting a substantial delay in repayments. Finally, the **total loan loss provisioning** is negative (-49,920,697,769.45), which may indicate calculation or data issues, as provision expenses are expected to be positive, reflecting potential credit losses. Overall, while defaults are low, the unusually high PAR, recovery, and negative provisioning figures warrant further investigation to ensure data accuracy and reliable portfolio risk assessment.
